In [ ]:
%load_ext autoreload
%autoreload 2

# Create database

In [ ]:
username = !whoami
username = username[0]
username

In [ ]:
import os
import datajoint as dj

config_file = f'/gpfs01/berens/user/{username}/datajoint/dj_{username}_conf.json'

assert os.path.isfile(config_file), 'Set the path to your config file'

In [ ]:
os.environ["DJ_SUPPORT_FILEPATH_MANAGEMENT"] = "TRUE"

In [ ]:
# Define a schema name or use the default name for your personal test schema
schema_name = f"ageuler_{username}_test"

In [ ]:
# Load configuration for user
dj.config.load(config_file)
dj.config['schema_name'] = schema_name
dj.config['enable_python_native_blobs'] = True
dj.config['stores'] = {"classifier": {"protocol": "file", "location": "/gpfs01/euler/data/Resources/Classifier", "stage": "/gpfs01/euler/data/Resources/Classifier"}}
print("schema_name:", dj.config['schema_name'])
dj.conn()

In [ ]:
if not os.path.isdir(dj.config['stores']['classifier']['stage']):
    raise FileNotFoundError('You need access to the classifier stage')

In [ ]:
if not os.path.isdir(dj.config['stores']['classifier']['location']):
    raise FileNotFoundError('You need access to the classifier location')

In [ ]:
from djimaging.schemas.core_schema import *
from djimaging.schemas.core_plus_optional_schema import *
from djimaging.schemas.rgc_classifier_schema import *

In [ ]:
from djimaging.utils.dj_utils import activate_schema

activate_schema(schema=schema, create_schema=True, create_tables=True)
schema

In [ ]:
from djimaging.tables.optional import rgc_classifier

In [ ]:
dj.ERD(schema)

# Add classifier method

In [ ]:
if len(ClassifierMethod()) == 0:
    classifier_fn = "sklearn.ensemble.RandomForestClassifier"
    classifier_config=dict(
        n_estimators=3000,
        class_weight="balanced",
        oob_score=True,
        n_jobs=12,
        random_state=0)

    ClassifierMethod().add_classifer(
        classifier_config=classifier_config,
        classifier_fn=classifier_fn, comment="test")

In [ ]:
ClassifierMethod()

# Add training data

In [ ]:
classifier_data_path = '/gpfs01/euler/data/Resources/Classifier/'
os.listdir(classifier_data_path)

In [ ]:
if len(ClassifierTrainingData()) == 0:
    ClassifierTrainingData().add_trainingdata(
        project="False",
        output_path=f'/gpfs01/berens/user/{username}/euler/datajoint/classifier',
        chirp_feats_file=classifier_data_path+'chirp_feats.npz',
        bar_feats_file=classifier_data_path+'bar_feats.npz',
        baden_data_file=classifier_data_path+'RGCData_postprocessed.mat',
        training_data_file=classifier_data_path+'training_all.pkl',
    )
ClassifierTrainingData()

# Make sure data is loaded to the DataBase

In [ ]:
userinfo = {
    'experimenter': 'DataJointTestData', # Replace this if you want to use your own data
    'data_dir': '/gpfs01/euler/data/Data/DataJointTestData/', # Replace this if you want to use your own data
    'datatype_loc': 0,
    'animal_loc': 1,
    'region_loc': 2,
    'field_loc': 3,
    'stimulus_loc': 4,
    'condition_loc': 5,
}

assert os.path.isdir(userinfo['data_dir'])

In [ ]:
UserInfo().upload_user(userinfo)
UserInfo()

In [ ]:
Experiment().rescan_filesystem(verboselvl=0)
Experiment()

In [ ]:
Field().rescan_filesystem(verboselvl=0)
Field()

In [ ]:
Roi().populate()
Roi()

In [ ]:
Field().populate()

In [ ]:
Stimulus().add_nostim(skip_duplicates=True)
Stimulus().add_chirp(spatialextent=1000, stim_name='gChirp', alias="chirp_gchirp_globalchirp", skip_duplicates=True)
Stimulus().add_chirp(spatialextent=300, stim_name='lChirp', alias="lchirp_localchirp", skip_duplicates=True)
Stimulus().add_noise(stim_name='noise', pix_n_x=20, pix_n_y=15, pix_scale_x_um=30, pix_scale_y_um=30, skip_duplicates=True)
Stimulus().add_movingbar(skip_duplicates=True)

In [ ]:
Presentation().populate()
Presentation()

In [ ]:
Traces().populate()
Traces()

In [ ]:
PreprocessParams().add_default(skip_duplicates=True)
PreprocessParams()

In [ ]:
PreprocessTraces().populate(display_progress=True)
PreprocessTraces()

In [ ]:
Snippets().populate(display_progress=True)
Snippets()

# Add ClassifierSeed and CellFilterParameters
if not already added

In [ ]:
if len(ClassifierSeed()) == 0:
    ClassifierSeed().insert1(dict(classifier_seed=8732))
ClassifierSeed()

In [ ]:
if len(CellFilterParameters()) == 0:
    CellFilterParameters().add_parameters(qi_thres_chirp=0.35, qi_thres_bar=0.6, cell_selection_constraint="or")
CellFilterParameters()

# Train classifier

In [ ]:
Classifier().populate()

In [ ]:
Classifier()

# Populate CelltypeAssignment

In [ ]:
pred_groups = CelltypeAssignment().fetch("predicted_group")

In [ ]:
sum(pred_groups>=0)

In [ ]:
from datetime import datetime
now = datetime.now()
date_time = now.strftime("%Y%m%d, %H:%M:%S")
print("date and time:",date_time)

In [ ]:
CelltypeAssignment().fetch("date")[0].strftime("%Y%m%d")

In [ ]:
#CelltypeAssignment().proj(neuron_id ='"_".join([date.replace("-", ""),exp_num, field_id, roi_id, s, str(5)])')
CelltypeAssignment().proj(date_str ='date.strftime("%Y%m%d")')

In [ ]:
CelltypeAssignment().proj(neuron_id ='CONCAT_WS("_", DATE_FORMAT(date, GET_FORMAT(DATE,"INTERNAL")), exp_num, field_id, roi_id, "s", "5")')

In [ ]:
CelltypeAssignment().proj(test ='DATE_SUB(date,INTERVAL 1 YEAR)')

In [ ]:
ClassifierTrainingData()

In [ ]:
ClassifierTrainingData().drop()